In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init as init

from IPython.display import clear_output

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    gpu_num = 1
    torch.cuda.set_device(gpu_num)
    torch.cuda.manual_seed_all(777)
    print(torch.cuda.get_device_properties(gpu_num).name, 'is Ready')

In [3]:
learning_rate = .001
training_epochs = 5
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test  = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

In [5]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [6]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.flatten = nn.Flatten()
        
        self.fc = nn.Linear(7*7*64, 10, bias=True)
        init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.flatten(out)
        out = self.fc(out)
        
        return out

In [7]:
model = CNN_Model().to(device)
model

CNN_Model(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten()
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [8]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
model.train()
total_batch = len(data_loader)

import time
s = time.time()
for epoch in range(training_epochs):
    avg_cost = 0
    
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)
        
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    # clear_output(wait=True)
    print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost))
    
print("Learning Finished")
print("Time Taken {:.6f}s by {}".format(time.time()-s, device))

[Epoch:    1] cost = 0.223937914
[Epoch:    2] cost = 0.0620741434
[Epoch:    3] cost = 0.0448859222
[Epoch:    4] cost = 0.0356147848
[Epoch:    5] cost = 0.029051125
Learning Finished
Time Taken 252.756209s by cpu


In [11]:
model.eval()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9855999946594238


In [11]:
model.state_dict()

OrderedDict([('layer1.0.weight',
              tensor([[[[-0.2847,  0.0263, -0.0059],
                        [-0.0308,  0.3160,  0.3241],
                        [ 0.2061,  0.0612,  0.2742]]],
              
              
                      [[[-0.2149, -0.0971,  0.1298],
                        [-0.2361,  0.1569,  0.2311],
                        [ 0.4090,  0.3588, -0.0739]]],
              
              
                      [[[ 0.3047,  0.3221,  0.3196],
                        [-0.1408, -0.0066, -0.1317],
                        [ 0.3348,  0.3306,  0.3071]]],
              
              
                      [[[-0.0763, -0.3314, -0.1929],
                        [-0.0334,  0.1300,  0.0744],
                        [ 0.4052,  0.0014,  0.3730]]],
              
              
                      [[[ 0.3429,  0.1797, -0.0768],
                        [ 0.1020, -0.1271, -0.1568],
                        [ 0.3253,  0.4010, -0.3728]]],
              
              
            

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [20]:
for p in model.parameters():
    print(p.shape)

torch.Size([32, 1, 3, 3])
torch.Size([32])
torch.Size([64, 32, 3, 3])
torch.Size([64])
torch.Size([10, 3136])
torch.Size([10])


In [17]:
count_parameters(model)

50186

In [14]:
torch.save(model, './CNN_Module.pt')

C:\Users\admin\workspace\Pytorch\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CNN_Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [15]:
model = torch.load('./CNN_Module.pt')

In [21]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, ceil_mode=True)
        )
        

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(4*4*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10)
        
        init.xavier_uniform_(self.fc1.weight)
        init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

In [22]:
model = CNN_Model().to(device)

In [24]:
print(model)

value = torch.Tensor(1, 1, 28, 28).to(device)
print(model(value).shape)

CNN_Model(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (flatten): Flatten()
  (fc1): Linear(in_features=2048, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)
torch.Size([1, 10])


In [25]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
def train():
    model.train()
    total_batch = len(data_loader)

    import time
    s = time.time()
    for epoch in range(training_epochs):
        avg_cost = 0

        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)

            hypothesis = model(x)
            cost = criterion(hypothesis, y)
            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch

        # clear_output(wait=True)
        print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost))

    print("Learning Finished")
    print("Time Taken {:.6f}s by {}".format(time.time()-s, device))

In [27]:
%timeit -n 1 -r 1 train()

[Epoch:    1] cost = 0.152267739
[Epoch:    2] cost = 0.0406920612
[Epoch:    3] cost = 0.0291638579
[Epoch:    4] cost = 0.0220566411
[Epoch:    5] cost = 0.0163107142
Learning Finished
Time Taken 460.846869s by cpu
7min 40s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [28]:
count_parameters(model)

1379557

In [29]:
model.eval()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/Users/honeyeob/python_workspace/Pytorch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/honeyeob/python_workspace/Pytorch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.987500011920929
